In [1]:
import torch
import torch.nn as nn

In [3]:
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix
def accuracies(y_true, y_pred, adaptive=True):
    y_true = np.array(y_true).flatten()
    y_pred = np.array(y_pred).flatten()
    auc = roc_auc_score(y_true, y_pred)

    if adaptive == True:
        fpr, tpr, thresholds = roc_curve(y_true, y_pred, pos_label=1)
        dist = fpr ** 2 + (1 - tpr) ** 2
        best_thres = thresholds[np.argmin(dist)]
    else:
        best_thres = 0.5

    y_pred_val = np.where(np.array(y_pred).flatten() >= best_thres, 1, 0)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred_val).ravel()
    accuracy = (tp + tn) / (tn + fp + fn + tp)
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    print('AUC score:', np.round(auc, 4))
    print('Accuracy:', np.round(accuracy, 4))
    print('Sensitivity:', np.round(sensitivity, 4))
    print('Specificity:', np.round(specificity, 4))

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
from tqdm import tqdm
from gru import GRUModel

torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

##############################################################
###################### Change this part ######################
X = np.load('data_X.npy')
y = np.load('data_y.npy')

#Imputation methods here 

X = np.nan_to_num(X, nan=1) # You do not have to do this after some traditional imputations
##############################################################

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
train_index, test_index = next(sss.split(X, y))
val_index, test_index = next(StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=0).split(X[test_index], y[test_index]))

train_dataset = TensorDataset(X_tensor[train_index], y_tensor[train_index])
val_dataset = TensorDataset(X_tensor[val_index], y_tensor[val_index])
test_dataset = TensorDataset(X_tensor[test_index], y_tensor[test_index])

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

input_size = 5
hidden_size = 16
output_size = 1
lr_rate = 0.001

model = GRUModel(input_size, hidden_size, output_size)
model.to(device)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)

early_stopping_patience = 20
early_stopping_counter = 0
best_val_loss = float('inf')
model_path = 'best_model_GRU.pth'

num_epochs = 1000
for epoch in tqdm(range(num_epochs)):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        predictions = model(inputs).squeeze()
        loss = criterion(predictions, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            predictions = model(inputs).squeeze()
            loss = criterion(predictions, labels)
            val_loss += loss.item() * inputs.size(0)

    train_loss = running_loss / len(train_loader.dataset)
    val_loss = val_loss / len(val_loader.dataset)

    print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')

    scheduler.step()

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stopping_counter = 0
        torch.save(model.state_dict(), model_path)
    else:
        early_stopping_counter += 1
        if early_stopping_counter >= early_stopping_patience:
            print("Early stopping triggered")
            break

model.load_state_dict(torch.load(model_path), map_location=torch.device('cuda'))

model.eval()
test_losses = []
test_labels = []
test_predictions = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs).squeeze()
        loss = criterion(outputs, labels)
        test_losses.append(loss.item())
        test_predictions.extend(outputs.tolist())
        test_labels.extend(labels.tolist())

average_test_loss = sum(test_losses) / len(test_losses)
print(f'Average test loss: {average_test_loss:.4f}')

test_predictions = np.array(test_predictions)
test_labels = np.array(test_labels)

accuracies(test_labels, test_predictions)

  0%|          | 1/1000 [00:08<2:22:37,  8.57s/it]

Epoch 1/1000, Training Loss: 0.5522, Validation Loss: 0.4279


  0%|          | 2/1000 [00:17<2:22:01,  8.54s/it]

Epoch 2/1000, Training Loss: 0.4419, Validation Loss: 0.4255


  0%|          | 3/1000 [00:25<2:22:45,  8.59s/it]

...

Epoch 5/1000, Training Loss: 0.4290, Validation Loss: 0.4131


  1%|          | 6/1000 [00:46<2:02:13,  7.38s/it]

Epoch 6/1000, Training Loss: 0.4182, Validation Loss: 0.3964


  1%|          | 6/1000 [00:53<2:27:24,  8.90s/it]


KeyboardInterrupt: 